In [1]:
import numpy as np
from itertools import groupby

In [2]:
sample_input = open('sample-input.txt')
puzzles = [line.strip() for line in [line for line in sample_input.read().splitlines() if len(line)!=0] if line[0]!='#']
puzzles

['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.',
 '..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..',
 'JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...',
 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4',
 'IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH. F0 G6',
 'BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']

In [3]:
def create_grid(puzzle:str):
    puzzle = puzzle[0:36]
    return np.array(list(puzzle), dtype=str).reshape(6,6)

def grid_to_string(current_grid_state:np.array):
    return ''.join(str(ele) for ele in current_grid_state.flatten())

def initial_fuel(puzzle:str):
    puzzle = puzzle[36:].split()
    return {car[0]:int(car[1]) for car in puzzle}
    

In [10]:
puzzle_grid = create_grid(puzzles[3])
print(puzzle_grid)

[['B' 'B' 'B' '.' '.' 'M']
 ['C' 'C' 'D' 'D' '.' 'M']
 ['A' 'A' 'K' 'L' '.' 'M']
 ['J' '.' 'K' 'L' 'E' 'E']
 ['J' '.' 'G' 'G' '.' '.']
 ['J' 'H' 'H' 'H' 'I' 'I']]


In [11]:
initial_fuel(puzzles[3])

{'J': 0, 'B': 4}

In [12]:
class Car():
    
    def __init__(self, is_horizontal:bool, letter:str, car_length:int, arr_indices, fuel):
        self.horizontal = is_horizontal
        self.letter = letter
        self.car_length = car_length
        self.arr_indices = arr_indices
        self.fuel = fuel
    
    def __str__(self):
        return "{direction} car '{letter}' with a length of {length}."\
        " Indices i:{indices_i} and j:{indices_j}. It has {fuel} fuel.".format(
            direction='Horizontal' if self.horizontal else 'Vertical',
            letter=self.letter,
            length=self.car_length,
            indices_i=self.arr_indices[0],
            indices_j=self.arr_indices[1],
            fuel=self.fuel)

In [17]:
def get_all_cars_in_grid(puzzle:np.array, fuel):
    
    #find horizontal cars
    cars = []
    for row in puzzle[range(6)]:
        #https://stackoverflow.com/a/6352456 for finding consecutive duplicates in a list
        grouped_row = [(letter, sum(1 for i in g)) for letter, g in groupby(row)]        
        grouped_row = [Car(True, letter, num, np.where(puzzle==letter), 100 if letter not in fuel else fuel[letter])
                       for letter, num in grouped_row if letter!='.' and num>1]
        if grouped_row is not None:
            cars = cars + grouped_row
    #find vertical cars
    for i in range(6):
        column = puzzle[:,i]
        grouped_column = [(letter, sum(1 for i in g)) for letter, g in groupby(column)]
        grouped_column = [Car(False, letter, num, np.where(puzzle==letter), 100 if letter not in fuel else fuel[letter])
                          for letter, num in grouped_column if letter!='.' and num>1]
        if grouped_column is not None:
            cars = cars + grouped_column
    
    fuel_list = {car.letter:car.fuel for car in cars}
    
    return cars, fuel_list

In [21]:
cars, fuel_list = get_all_cars_in_grid(puzzle_grid, initial_fuel(puzzles[3]))

In [23]:
for car in cars:
    print(car)

Horizontal car 'B' with a length of 3. Indices i:[0 0 0] and j:[0 1 2]. It has 4 fuel.
Horizontal car 'C' with a length of 2. Indices i:[1 1] and j:[0 1]. It has 100 fuel.
Horizontal car 'D' with a length of 2. Indices i:[1 1] and j:[2 3]. It has 100 fuel.
Horizontal car 'A' with a length of 2. Indices i:[2 2] and j:[0 1]. It has 100 fuel.
Horizontal car 'E' with a length of 2. Indices i:[3 3] and j:[4 5]. It has 100 fuel.
Horizontal car 'G' with a length of 2. Indices i:[4 4] and j:[2 3]. It has 100 fuel.
Horizontal car 'H' with a length of 3. Indices i:[5 5 5] and j:[1 2 3]. It has 100 fuel.
Horizontal car 'I' with a length of 2. Indices i:[5 5] and j:[4 5]. It has 100 fuel.
Vertical car 'J' with a length of 3. Indices i:[3 4 5] and j:[0 0 0]. It has 0 fuel.
Vertical car 'K' with a length of 2. Indices i:[2 3] and j:[2 2]. It has 100 fuel.
Vertical car 'L' with a length of 2. Indices i:[2 3] and j:[3 3]. It has 100 fuel.
Vertical car 'M' with a length of 3. Indices i:[0 1 2] and j:[5

In [24]:
fuel_list

{'B': 4,
 'C': 100,
 'D': 100,
 'A': 100,
 'E': 100,
 'G': 100,
 'H': 100,
 'I': 100,
 'J': 0,
 'K': 100,
 'L': 100,
 'M': 100}

1. Group the consecutive duplicate letters
2. Find the neighbours of the car's letter.
3. Append a tuple of the group of empty spaces with the array index adjacent to the car to a list if it neighbours the car
4. Return this list

Front spaces are defined as j+1 or i+1. AKA going down or going to the right of the matrix.
Back spaces are defined as j-1 or i-1. AKA going up or going to the left of the matrix.

In [ ]:
def find_available_spaces_for_car_on_grid(car:Car, puzzle_grid:np.array):
    
    back_spaces = 0
    front_spaces = 0
    
    if car.horizontal:
        relevant_groups = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[car.arr_indices[0][1],:])]
        index_of_car=[i for i, group in enumerate(relevant_groups) if group[0]==car.letter][0]
    else:
        relevant_groups = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[:,car.arr_indices[1][0]])]
        index_of_car=[i for i, group in enumerate(relevant_groups) if group[0]==car.letter][0]
    
    try:
        front_neighbour = relevant_groups[index_of_car+1]
        front_spaces = front_neighbour[1] if front_neighbour[0]=='.' else 0
    except IndexError:
        front_spaces = 0
    try:
        back_neighbour = relevant_groups[index_of_car-1] if index_of_car > 0 else ('',0)
        back_spaces = back_neighbour[1] if back_neighbour[0]=='.' else 0
    except IndexError:
        back_spaces = 0
    
    return car, back_spaces, front_spaces

In [ ]:
car_moves = [find_available_spaces_for_car_on_grid(car, puzzle_grid) for car in cars]
car_moves = filter(lambda moves:moves[1]!=0 or moves[2]!=0, car_moves)
car_moves = list(car_moves)
print(car_moves)

In [ ]:
car_moves[0][0].letter

1. Pass car_moves into move_car? Let's assume that the passed amount is valid for the puzzle
2. Update the car's array indices
3. Update the grid/empty space, replace the car's old position with '.'s, then place the car at its new position
4. return new puzzle grid

In [ ]:
def goal_state(puzzle_grid:np.array):
    return puzzle_grid[2][5]=='A'

In [ ]:
def move_car(car:Car, puzzle_grid:np.array, amount:int):
    #Replace car with empty space
    new_grid = np.copy(puzzle_grid)
    new_grid[new_grid==car.letter] = '.'
    
    indices_i, indices_j = car.arr_indices
    
    if car.horizontal:
        indices_j = indices_j + amount
    else:
        indices_i = indices_i + amount
    
    #Put car in new array indices
    for x in range(len(car.arr_indices[0])):
        new_grid[indices_i[x],indices_j[x]] = car.letter
            
    return new_grid

In [ ]:
def remove_car(puzzle_grid:np.array):
    if can_remove_car(puzzle_grid):
        letter = puzzle_grid[2][5]
        new_grid = np.copy(puzzle_grid)
        new_grid[new_grid==letter] = '.'
        return new_grid, letter
    else:
        return None, ''

In [ ]:
def can_remove_car(puzzle_grid:np.array):
    
    if goal_state(puzzle_grid):
        return False
    
    goal_row = puzzle_grid[2,:]
    
    car_groups = [(letter, sum(1 for i in g)) for letter, g in groupby(goal_row)]
    
    return car_groups[-1][0]!='.' and car_groups[-1][1]>1

In [ ]:
can_remove_car(puzzle_grid)

In [ ]:
def save_state(puzzle_grid:np.array):
    saved_grid = np.copy(puzzle_grid)
    return saved_grid

In [16]:
def explore_all_states(current_grid_state:np.array):
    new_states = []
    # Find move state where a car is removed
    remove_car_state, removed_car_letter = remove_car(current_grid_state)
    if remove_car_state is not None:
        new_states.append((remove_car_state, removed_car_letter, 0))
    # Get all moves
    cars = get_all_cars_in_grid(current_grid_state)
    car_moves = [find_available_spaces_for_car_on_grid(car, current_grid_state) for car in cars]
    car_moves = list(filter(lambda moves:moves[1]!=0 or moves[2]!=0, car_moves))
    # Create all move states
    
    for move in car_moves:
        current_car, back_spaces, front_spaces = move
        if back_spaces>0:
            for i in range(1,back_spaces+1):
                if(current_car.fuel-i>=0):
                    moved_car_state = move_car(current_car, current_grid_state, -i)
                    new_states.append((moved_car_state, current_car.letter, -i))
        if front_spaces>0:
            for i in range(1,front_spaces+1):
                if(current_car.fuel-i>=0):
                    moved_car_state = move_car(current_car, current_grid_state, i)
                    new_states.append((moved_car_state, current_car.letter, i))
    return new_states

In [ ]:
new_states = explore_all_states(puzzle_grid)
new_states

In [ ]:
def blocking_heuristic(puzzle_grid:np.array):
    #since we always know 'A' car is on the third row, we just make group the duplicate characters on the third row
    row = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[2])]
    #filter out the empty space since we can ignore it the heuristic
    row = list(filter(lambda group: group[0]!='.', row))
    #find the index of the 'A' in the group list
    index_of_car=[i for i, group in enumerate(row) if group[0]=='A'][0]+1
    #return length of group list - 'A' car's index to get how many cars are blocking 'A'
    return len(row)-index_of_car

In [ ]:
def blocked_positions_heuristic(puzzle_grid:np.array):
    #since we always know 'A' car is on the third row, we just make group the duplicate characters on the third row
    row = [(letter, sum(1 for i in g)) for letter, g in groupby(puzzle_grid[2])]
    #filter out the empty space
    row = list(filter(lambda group: group[0]!='.', row))
    #find the index of the 'A' in the group list
    index_of_car=[i for i, group in enumerate(row) if group[0]=='A'][0]+1
    #sum all the groups after 'A'
    blocked_positions = sum(blocking_groups[1] for blocking_groups in row[index_of_car:])
    return blocked_positions

In [ ]:
#a* algorithm from https://leetcode.com/problems/shortest-path-in-binary-matrix/discuss/313347/a-search-in-python

from heapq import heappush, heappop

class PriorityQueue:
    
    def __init__(self, iterable=[]):
        self.heap = []
        for value in iterable:
            heappush(self.heap, (0, value))
    
    def add(self, value, priority=0):
        heappush(self.heap, (priority, value))
    
    def pop(self):
        priority, value = heappop(self.heap)
        return value
    
    def __len__(self):
        return len(self.heap)

In [ ]:
#a* algorithm from https://leetcode.com/problems/shortest-path-in-binary-matrix/discuss/313347/a-search-in-python

def a_star_search(
        start,
        goal,
        successors,
        heuristic
    ):
    start = grid_to_string(start)
    visited = set()
    came_from = dict()
    distance = {start:0}
    frontier = PriorityQueue()
    frontier.add(start)
    while frontier:
        node = frontier.pop()
        if node in visited:
            continue
        if goal(create_grid(node)):
            return reconstruct_path(came_from, start, node)
        visited.add(node)
        for successor in successors(create_grid(node)):
            successor_state, successor_letter, successor_distance = successor
            state_string = grid_to_string(successor_state)
            successor_distance = abs(successor_distance)
            
            frontier.add(state_string, priority=distance[node] + successor_distance + heuristic(successor_state))
            if (state_string not in distance
                or distance[node] + successor_distance < distance[state_string]):
                distance[state_string] = distance[node] + successor_distance
                came_from[state_string] = node
    return None

In [ ]:
def reconstruct_path(came_from, start, end):
    reverse_path = [end]
    while end != start:
        end = came_from[end]
        reverse_path.append(end)
    return list(reversed(reverse_path))

In [ ]:
shortest_path = a_star_search(puzzle_grid, goal_state, explore_all_states, blocking_heuristic)

In [ ]:
print(grid_to_string(puzzle_grid))

In [ ]:
shortest_path

In [ ]:
grids = [create_grid(state) for state in shortest_path]

In [ ]:
grids